In [1]:
import pandas as pd

df = pd.read_csv('df_gpt4.csv')

df = df.dropna(subset=['cleaned text'])

df

,Institution name,Main panel_x,Unit of assessment number_x,Unit of assessment name,Title,Summary impact type,Countries,Formal partners,Funding programmes,Global research identifiers,...,1. Summary of the impact,2. Underpinning research,3. References to the research,4. Details of the impact,5. Sources to corroborate the impact,Weighted Avg,Variance,5 Class Labels,9 Class Labels,cleaned text
0,University of East London,C,21.0,Sociology,"?Go Home?; Shaping the practice, policy and un...",Societal,[Italy];[Sweden];[Netherlands ];[Ireland];[Uni...,[University of Eastern Finland];[Middle East T...,[European Commission under the 7th Framework P...,NaN,...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,2.667,0.556111,3,5,"Since 2013, CMRB findings have been used to su..."
1,University of Lincoln,A,6.0,"Agriculture, Food and Veterinary Sciences",Agricultural Robotics: Data Driven Technologie...,Technological,[England];[Norway];[USA],[SAGA Robotics];[Garford Farms Limited];[Berry...,[N/A],[grid.423443.6];[grid.418100.c],...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,3.000,0.666000,3,6,UoL’s innovative agri-food research has underp...
2,University of Cambridge,A,4.0,"Psychology, Psychiatry and Neuroscience",Alleviating the societal and economic burden o...,Health,[England];[China];[USA];[Canada];[Belgium];[Fr...,NaN,NaN,[grid.14105.31];[grid.473755.7];[grid.52788.30],...,**1. Summary of the impact** \(indicative maxi...,**2. Underpinning research \(indicative maximu...,**3. References to the research** \(indicative...,**4. Details of the impact** \(indicative maxi...,**5. Sources to corroborate the impact** \(ind...,3.667,0.444111,4,7,The Cambridge University research has had a si...
3,University of Cambridge,A,6.0,"Agriculture, Food and Veterinary Sciences",Changes in policy for the control of bovine tu...,Environmental,[England];[Wales];[Ethiopia],NaN,NaN,[grid.418100.c];[grid.434257.3];[grid.433527.4...,...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,3.500,0.450000,4,7,Bovine tuberculosis is a substantial economic ...
4,University of Derby,C,20.0,Social Work and Social Policy,Changing Police and Investigative Practice thr...,Legal,NaN,NaN,NaN,NaN,...,### 1. Summary of the impact \(indicative max...,### 2. Underpinning research \(indicative max...,### 3. References to the research \(indicativ...,### 4. Details of the impact \(indicative max...,### 5. Sources to corroborate the impact \(in...,2.333,1.554111,2,5,The impact of Bull's work is on: (1) internati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6293,School of Oriental and African Studies,C,22.0,Anthropology and Development Studies,Transforming Working Practices in the House of...,Societal,"[UK, Europe, Asia, Africa]",[Hansard Society];[Leeds University];[Jawaharl...,[ERC-2018-ADG - ERC Advanced Grant];[Research ...,[grid.452896.4];[grid.452966.a];[grid.426413.6...,...,**1. Summary of the impact** \(indicative maxi...,**2. Underpinning research** \(indicative maxi...,**3. References to the research** \(indicative...,**4. Details of the impact** \(indicative maxi...,**5. Sources to corroborate the impact** \(ind...,3.750,0.187500,4,8,"Many scholars, MPs, civil servants and Commons..."
6300,Nottingham Trent University,A,3.0,"Allied Health Professions, Dentistry, Nursing ...",Translating biomedical nanotechnology into com...,Technological,"[Algeria,

In [2]:
import pandas as pd
import tensorflow
from nltk.tokenize import regexp_tokenize, sent_tokenize
from nltk import pos_tag_sents
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')


#Dictionary of contractions for replacement 

contractions = {
          "ain't": "am not",
          "aren't": "are not",
          "can't": "cannot",
          "can't've": "cannot have",
          "'cause": "because",
          "could've": "could have",
          "couldn't": "could not",
          "couldn't've": "could not have",
          "didn't": "did not",
          "doesn't": "does not",
          "don't": "do not",
          "hadn't": "had not",
          "hadn't've": "had not have",
          "hasn't": "has not",
          "haven't": "have not",
          "he'd": "he would",
          "he'd've": "he would have",
          "he'll": "he will",
          "he'll've": "he will have",
          "he's": "he is",
          "how'd": "how did",
          "how'd'y": "how do you",
          "how'll": "how will",
          "how's": "how is",
          "i'd": "i would",
          "i'd've": "i would have",
          "i'll": "i will",
          "i'll've": "i will have",
          "i'm": "i am",
          "i've": "i have",
          "isn't": "is not",
          "it'd": "it had",
          "it'd've": "it would have",
          "it'll": "it will",
          "it'll've": "it will have",
          "it's": "it is",
          "let's": "let us",
          "ma'am": "madam",
          "mayn't": "may not",
          "might've": "might have",
          "mightn't": "might not",
          "mightn't've": "might not have",
          "must've": "must have",
          "mustn't": "must not",
          "mustn't've": "must not have",
          "needn't": "need not",
          "needn't've": "need not have",
          "o'clock": "of the clock",
          "oughtn't": "ought not",
          "oughtn't've": "ought not have",
          "shan't": "shall not",
          "sha'n't": "shall not",
          "shan't've": "shall not have",
          "she'd": "she would",
          "she'd've": "she would have",
          "she'll": "she will",
          "she'll've": "she will have",
          "she's": "she is",
          "should've": "should have",
          "shouldn't": "should not",
          "shouldn't've": "should not have",
          "so've": "so have",
          "so's": "so is",
          "that'd": "that would",
          "that'd've": "that would have",
          "that's": "that is",
          "there'd": "there had",
          "there'd've": "there would have",
          "there's": "there is",
          "they'd": "they would",
          "they'd've": "they would have",
          "they'll": "they will",
          "they'll've": "they will have",
          "they're": "they are",
          "they've": "they have",
          "to've": "to have",
          "wasn't": "was not",
          "we'd": "we had",
          "we'd've": "we would have",
          "we'll": "we will",
          "we'll've": "we will have",
          "we're": "we are",
          "we've": "we have",
          "weren't": "were not",
          "what'll": "what will",
          "what'll've": "what will have",
          "what're": "what are",
          "what's": "what is",
          "what've": "what have",
          "when's": "when is",
          "when've": "when have",
          "where'd": "where did",
          "where's": "where is",
          "where've": "where have",
          "who'll": "who will",
          "who'll've": "who will have",
          "who's": "who is",
          "who've": "who have",
          "why's": "why is",
          "why've": "why have",
          "will've": "will have",
          "won't": "will not",
          "won't've": "will not have",
          "would've": "would have",
          "wouldn't": "would not",
          "wouldn't've": "would not have",
          "y'all": "you all",
          "y'alls": "you alls",
          "y'all'd": "you all would",
          "y'all'd've": "you all would have",
          "y'all're": "you all are",
          "y'all've": "you all have",
          "you'd": "you had",
          "you'd've": "you would have",
          "you'll": "you you will",
          "you'll've": "you you will have",
          "you're": "you are",
          "you've": "you have"
            }

2024-08-28 00:46:31.917376: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 00:46:31.920218: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 00:46:31.968271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 00:46:31.969036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 00:46:33.228950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
import string 

from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
stopwords_set = set(stopwords.words('english'))
punctuation_set = set(string.punctuation)


def penn_to_wordnet(penn_pos_tag):
    """
    Function to convert Penn Treebank part-of-speech tags to corresponding WordNet tags.
    """
    tag_dictionary = {
        'NN': 'n', 'NNS': 'n', 'NNP': 'n', 'NNPS': 'n',
        'JJ': 'a', 'JJR': 'a', 'JJS': 'a',
        'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v',
        'RB': 'r', 'RBR': 'r', 'RBS': 'r',
        'MD': 'v'
    }

    # Return the WordNet tag for given Penn Treebank tag, return "n" if none found
    return tag_dictionary.get(penn_pos_tag[:2], "n")

def preprocess_text(text):
    """
    Function to preprocess text: lowercasing, removing punctuation, expanding contractions, lemmatization, etc.
    """
    # Account for edge case when text is empty
    if not isinstance(text, str):
        return ""

    # Convert all text to lowercase
    text = text.lower()

    # Remove characters that are repeated more than twice in a row
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Replace URLs with space
    text = re.sub(r"http\S+", " ", text)

    # Remove HTML tags from text
    text = re.sub("<.*?>", "", text)

    # Ensure that there is space after each period
    text = re.sub(r'\.(?![ .])', '. ', text)

    # Replace line break tags with space
    text = re.sub(r'<br />', ' ', text)

    # Remove words containing digits
    text = re.sub(r'\d+', '', text)

    # Remove all numbers
    text = re.sub(r'\b(?=.*\d)(?=.*[a-zA-Z]).*?\b', '', text)

    # Remove all numbers including floating points
    text = re.sub(r'\b\d+(\.\d+)?', '', text)

    # Tokenize sentences and words
    tokenized_sentences = [regexp_tokenize(sent, pattern="\s+", gaps=True) for sent in sent_tokenize(text)]

    new_sentences = []

    # Use tqdm to visualize the progress of processing sentences
    for sentences in pos_tag_sents(tokenized_sentences):
        new_sentence = []
        # Iterate through word/tag pair
        for word, tag in sentences:
            # Check if word in punctuation and stopwords list
            if word not in punctuation_set and word not in stopwords_set:
                # Expand contractions if present
                if word in contractions:
                    words = contractions[word].split(" ")
                    new_sentence.extend(words)
                else:
                    # Get WordNet Pos Tag and lemmatize word
                    wordnet_pos = penn_to_wordnet(tag)
                    lemmatized_word = wnl.lemmatize(word, pos=wordnet_pos)
                    # Remove non-alphanumeric characters
                    lemmatized_word = re.sub(r'[^\w\s]', '', lemmatized_word)

                    # Check if lemmatized word exists and has length greater than 2
                    if lemmatized_word and len(lemmatized_word) > 2:
                        new_sentence.append(lemmatized_word)

        if new_sentence:
            new_sentences.append(new_sentence)

    return new_sentences

In [4]:
tqdm.pandas()


new_df = df[["Main panel_x","Unit of assessment number_x","Title","cleaned text","5 Class Labels","Summary impact type","9 Class Labels", "Variance", "Weighted Avg"]]

new_df = new_df[new_df["Main panel_x"] == "A"]


new_df["sentence tokenized"] = new_df["cleaned text"].apply(lambda x: preprocess_text(x))



In [5]:
new_df

,Main panel_x,Unit of assessment number_x,Title,cleaned text,5 Class Labels,Summary impact type,9 Class Labels,Variance,Weighted Avg,sentence tokenized
1,A,6.0,Agricultural Robotics: Data Driven Technologie...,UoL’s innovative agri-food research has underp...,3,Technological,6,0.666000,3.000,"[[uols, innovative, agrifood, research, underp..."
2,A,4.0,Alleviating the societal and economic burden o...,The Cambridge University research has had a si...,4,Health,7,0.444111,3.667,"[[cambridge, university, research, significant..."
3,A,6.0,Changes in policy for the control of bovine tu...,Bovine tuberculosis is a substantial economic ...,4,Environmental,7,0.450000,3.500,"[[bovine, tuberculosis, substantial, economic,..."
7,A,2.0,Empowering personalised discussion and advance...,Advance decisions about resuscitation are a gl...,4,Health,7,0.358124,3.626,"[[advance, decision, resuscitation, global, he..."
17,A,5.0,Improving patient outcomes and treatment guide...,Professor Irving's research has been instrumen...,3,Health,6,0.454204,3.214,"[[professor, irvings, research, instrumental, ..."
...,...,...,...,...,...,...,...,...,...,...
6262,A,4.0,Transforming the use of Arterial Spin Labellin...,The Arterial Spin Labelling In Dementia (AID) ...,4,Technological,7,0.655264,3.644,"[[arterial, spin, labelling, dementia, aid, pr..."
6271,A,5.0,Transforming therapeutic outcomes of fatal Duc...,Research undertaken at RHUL led to the develop...,3,Health,7,0.222111,3.333,"[[research, undertaken, rhul, lead, developmen..."
6275,A,3.0,Transforming Treatment for Obstructive Sleep A...,Obstructive sleep apnoea (OSA) is one of the m...,3,Health,7,0.234375,3.375,"[[obstructive, sleep, apnoea, osa, one, common..."
6289,A,1.0,Transforming vaccine policy for pneumococcal d...,It was the result of Professor Lim's expertise...,3,Health,7,0.334111,3.333,"[[result, professor, lims, expertise, field, l..."


In [6]:
def flatten(sents_tokenized):
    return [word for sentence in sents_tokenized for word in sentence]

def tokens_to_string(tokens):
    return ' '.join(tokens)

    
new_df["word tokenized"] = new_df["sentence tokenized"].apply(lambda x: flatten(x))

new_df["string"] = new_df["word tokenized"].apply(lambda x: tokens_to_string(x))

In [7]:
def map_labels(label):
    if label in [0, 1, 2, 3, 4, 5]:
        return 0
    elif label in [6, 7]:
        return 1
    elif label == 8:
        return 2
    else:
        return label
    
new_df['3 Class Labels'] = new_df['9 Class Labels'].apply(map_labels)


In [8]:
result_df = new_df

In [9]:
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True)



In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [11]:
labels = result_df["3 Class Labels"]
texts = result_df["cleaned text"]

train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.3, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)

train_df = pd.DataFrame({'text': train_texts, 'label': train_labels})
val_df = pd.DataFrame({'text': val_texts, 'label': val_labels})
test_df = pd.DataFrame({'text': test_texts, 'label': test_labels})

train_df

,text,label
93,The research of Wormstone and team in developi...,1
2617,Driving conservation efforts through establish...,1
874,Bariatric surgery is a widely used interventio...,0
2747,Perinatal depression can have devastating cons...,2
5719,Professor Stallard's research has improved the...,1
...,...,...
4785,Estimates of the burden of viral hepatitis wer...,1
5026,The landscape of the emergency care workforce ...,1
6027,Details of the impact. New diagnostic testing ...,1
3891,Details of the impact\n\nHaving identified a c...,1


In [12]:
from sklearn.utils import shuffle
import nlpaug.augmenter.word as naw


def augment_text(df, samples, class_id):
    contextual_aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action='substitute', device='cuda')

    new_texts = []
    
    df_minority = df[df["label"] == class_id].reset_index(drop=True)
    
    for i in tqdm(np.random.randint(0, len(df_minority), samples)):
        text = df_minority.iloc[i]['text']
        augmented_text = contextual_aug.augment(text)
        new_texts.append(augmented_text[0])
    
    new_df = pd.DataFrame({'text': new_texts, 'label': class_id})

    df = shuffle(pd.concat([df, new_df]).reset_index(drop=True))
    
    return df

augmented_df = augment_text(train_df, 200, 0)

augmented_df = augment_text(augmented_df, 250, 2)



/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated

In [13]:
augmented_df["label"].value_counts()

1    615
2    467
0    313
Name: label, dtype: int64

In [14]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(augmented_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset =  Dataset.from_pandas(val_df)


dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': val_dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1395
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 203
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 202
    })
})


In [15]:
import pandas as pd
import datasets
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')



Using device: cuda


/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
text_encoded = dataset_dict.map(tokenization, batched=True, batch_size=None)

text_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map: 100%|██████████| 202/202 [00:00<00:00, 528.60 examples/s]


In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
from itertools import product
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

   
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    
   
    acc = accuracy_score(labels, preds)

    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

param_grid = {
    "per_gpu_batch_size": [4, 8],
    "learning_rate": [2e-5, 3e-5, 5e-5],
    "num_epochs": [2, 3, 4],
    "weight_decay": [0.0, 0.01, 0.1, 0.0001, 0.001],  # Added weight decay values
    "warmup_steps": [0, 100, 200, 500, 1000]  # Added warmup steps values
}

# Get all combinations of hyperparameters
param_combinations = list(product(
    param_grid['per_gpu_batch_size'],
    param_grid['learning_rate'],
    param_grid['num_epochs'],
    param_grid['weight_decay'],
    param_grid['warmup_steps']
))

results = []

best_eval_result = None
best_params = None
best_f1 = 0.0  

for per_gpu_batch_size, learning_rate, num_epochs, weight_decay, warmup_steps in param_combinations:
    print(f"Training with batch size {per_gpu_batch_size}, learning rate {learning_rate}, num_epochs {num_epochs}, weight_decay {weight_decay}, warmup_steps {warmup_steps}")
    
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

    model.to(device)    
    training_args = TrainingArguments(
        output_dir='./results',          
        num_train_epochs=num_epochs,              
        per_device_train_batch_size=per_gpu_batch_size,   
        evaluation_strategy="epoch",     
        save_strategy="epoch",  
        fp16=True,  
        gradient_accumulation_steps=4, 
        save_total_limit=1,              
        load_best_model_at_end=True,     
        learning_rate=learning_rate,              
        weight_decay=weight_decay,              
        logging_dir='./logs',            
        logging_steps=10,
        report_to="none",                
        warmup_steps=warmup_steps  
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=text_encoded['train'],
        eval_dataset=text_encoded['validation'],
        compute_metrics=compute_metrics  # This function should return a dictionary with 'f1' key for F1 score
    )

    trainer.train()

    eval_result = trainer.evaluate()

    print(f"Evaluation results: {eval_result}")

    results.append({
        "per_gpu_batch_size": per_gpu_batch_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs,
        "weight_decay": weight_decay,
        "warmup_steps": warmup_steps,
        "f1": eval_result['eval_f1'],  
        "acc": eval_result['eval_accuracy']  
    })

    torch.cuda.empty_cache()

    if eval_result['eval_f1'] > best_f1:
        best_f1 = eval_result['eval_f1']
        best_eval_result = eval_result
        best_params = {
            "per_gpu_batch_size": per_gpu_batch_size,
            "learning_rate": learning_rate,
            "num_epochs": num_epochs,
            "weight_decay": weight_decay,
            "warmup_steps": warmup_steps
        }
        model.save_pretrained('./best_model')
        tokenizer.save_pretrained('./best_tokenizer')

    print(f"Current best params: {best_params} with F1 score: {best_f1}")

print(f"Best hyperparameters found: {best_params}")
print(f"Best evaluation results: {best_eval_result}")


Training with batch size 4, learning rate 2e-05, num_epochs 2, weight_decay 0.0, warmup_steps 0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfigurat

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.948400,0.870236,0.569307,0.551040,0.579036,0.569307
1,0.779000,0.837933,0.599010,0.600558,0.613692,0.599010


Evaluation results: {'eval_loss': 0.8379329442977905, 'eval_accuracy': 0.599009900990099, 'eval_f1': 0.6005578484472754, 'eval_precision': 0.6136918097045466, 'eval_recall': 0.599009900990099, 'eval_runtime': 2.8532, 'eval_samples_per_second': 70.797, 'eval_steps_per_second': 9.113, 'epoch': 1.99}
Current best params: {'per_gpu_batch_size': 4, 'learning_rate': 2e-05, 'num_epochs': 2, 'weight_decay': 0.0, 'warmup_steps': 0} with F1 score: 0.6005578484472754
Training with batch size 4, learning rate 2e-05, num_epochs 2, weight_decay 0.0, warmup_steps 100


/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home2/xtdm45/mds-project/topic_modelling_2/lib/python3.8/site-pack

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [4]:
import pandas as pd

df_results

df_results.to_csv('hyperparameter_results.csv', index=False)

df_results

,per_gpu_batch_size,learning_rate,num_epochs,f1,acc
0,4,0.00002,2,0.596149,0.594059
1,4,0.00002,3,0.506367,0.504950
2,4,0.00002,4,0.618383,0.638614
3,4,0.00003,2,0.577290,0.584158
4,4,0.00003,3,0.585873,0.584158
5,4,0.00003,4,0.457015,0.470297
6,4,0.00005,2,0.529366,0.529703
7,4,0.00005,3,0.585575,0.599010
8,4,0.00005,4,0.564446,0.574257
9,8,0.00002,2,0.572737,0.569307


In [5]:
import pandas as pd

df_results = pd.read_csv('hyperparameter_results.csv')

print(df_results)

    per_gpu_batch_size  learning_rate  num_epochs        f1       acc
0                    4        0.00002           2  0.596149  0.594059
1                    4        0.00002           3  0.506367  0.504950
2                    4        0.00002           4  0.618383  0.638614
3                    4        0.00003           2  0.577290  0.584158
4                    4        0.00003           3  0.585873  0.584158
5                    4        0.00003           4  0.457015  0.470297
6                    4        0.00005           2  0.529366  0.529703
7                    4        0.00005           3  0.585575  0.599010
8                    4        0.00005           4  0.564446  0.574257
9                    8        0.00002           2  0.572737  0.569307
10                   8        0.00002           3  0.554317  0.549505
11                   8        0.00002           4  0.588892  0.608911
12                   8        0.00003           2  0.543628  0.539604
13                  